In [ ]:
import pandas as pd
import os
import librosa.display
import ast
import soundfile as sf
# Function to perform Voice Activity Detection (VAD)
from Model.Preprocessing import Preprocessing
import json


In [2]:
ROOT_FOLDER = ''
folder_path = os.path.join(ROOT_FOLDER, 'data/Rec/')
audio_files = os.listdir(folder_path)
Transcript_df = pd.read_csv(os.path.join(ROOT_FOLDER, 'data/Transcription_data_from_mihub.csv'))

In [3]:
print(Transcript_df.shape)
print(Transcript_df['audio_name'].nunique())
temp_list = list(Transcript_df['audio_name'])

(956, 7)
956


In [4]:
cnt_process = 0
transcript_json = {}
for filename in audio_files:
    if cnt_process%50 == 0:
        print(cnt_process)
    if filename in temp_list:
        try:
            Pre_class = Preprocessing(audio_path = os.path.join(ROOT_FOLDER,'data/Rec/',filename))
            Pre_class.separate_channels()
            lc,rc, sr = Pre_class.extract_segments()

            trans = ast.literal_eval(Transcript_df[Transcript_df['audio_name'] == filename]['transcription'].iloc[0])
            speaker = ast.literal_eval(Transcript_df[Transcript_df['audio_name'] == filename]['speaker'].iloc[0])

            # if (len(lc)/sr <= 5) or (len(rc)/sr <= 5) or (len(trans) != len(speaker)):
            #     continue
            
            segment_path = os.path.join(ROOT_FOLDER,'data/Processed_Files',f"{filename}_lc.wav")
            sf.write(segment_path, lc, sr)
            segment_path = os.path.join(ROOT_FOLDER,'data/Processed_Files',f"{filename}_rc.wav")
            sf.write(segment_path, rc, sr)
            trans_lc = []#agent 
            trans_rc = []#customer
            for i in range(len(trans)):
                if speaker[i] == 'agent':
                    trans_lc.append(trans[i])
                else:
                    trans_rc.append(trans[i])
            
            transcript_json[f"{filename}_lc.wav"]  = " ".join(trans_lc)
            transcript_json[f"{filename}_rc.wav"]  = " ".join(trans_rc)
            cnt_process += 1
        except:
            pass

save_path = os.path.join(ROOT_FOLDER, 'data/Processed_Files', "Transcript.json")
with open(save_path, "w", encoding="utf-8") as f:
    json.dump(transcript_json, f, ensure_ascii=False, indent=4)


0
50
50
100
150
200
250
300
350
400
450
500
500
550
600
650
700
750
800


In [5]:
print(cnt_process)

813


In [ ]:
# Read JSON file
with open(os.path.join(ROOT_FOLDER, 'data/Processed_Files', "Transcript.json"), "r", encoding="utf-8") as f:
    transcript_json = json.load(f)
file = '20241208090446_9873656524.mp3_lc.wav'

In [ ]:
audio, sr = librosa.load(os.path.join(ROOT_FOLDER,'data/Processed_Files/',file), mono=True, sr=16000)

In [ ]:
import torchaudio
import matplotlib.pyplot as plt
from IPython.display import Audio
display(Audio(audio, rate=sr))

In [ ]:
transcript_json[file]